In [16]:
import boto3
import pandas as pd
from io import StringIO

# S3 client
s3_client = boto3.client('s3')

#  S3 bucket and file paths
bucket_name = 'nexttrendco'
file_path = 'rossman/train.csv'

# download  file from S3
response = s3_client.get_object(Bucket=bucket_name, Key=file_path)

# read CSV from the response into a pandas DataFrame
train_rossmann = pd.read_csv(response['Body'])


/tmp/ipykernel_125/1507509432.py:16: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  train_rossmann = pd.read_csv(response['Body'])


In [23]:
# convert 'Date' column to datetime format
train_rossmann['Date'] = pd.to_datetime(train_rossmann['Date'], errors='coerce')

# fill missing values for numeric columns with median
numeric_columns = train_rossmann.select_dtypes(include=['number']).columns
train_rossmann[numeric_columns] = train_rossmann[numeric_columns].fillna(train_rossmann[numeric_columns].median())

# fill missing categorical columns with mode 
categorical_columns = train_rossmann.select_dtypes(include='object').columns
train_rossmann[categorical_columns] = train_rossmann[categorical_columns].fillna(train_rossmann[categorical_columns].mode().iloc[0])

print("Missing values handled successfully")


Missing values handled successfully


In [26]:
# Lag Features
train_rossman['lag_1'] = train_rossmann['Sales'].shift(1)
train_rossman['lag_7'] = train_rossmann['Sales'].shift(7)

# Rolling Mean Features
train_rossman['rolling_mean_7'] = train_rossmann['Sales'].rolling(window=7).mean()

# Interaction Features
train_rossman['promo_holiday_interaction'] = train_rossmann['Promo'] * train_rossmann['StateHoliday']

# Convert the 'Date' column to datetime
train_rossman['Date'] = pd.to_datetime(train_rossmann['Date'])

# Extract day of week and month from the 'Date' column
train_rossman['day_of_week'] = train_rossmann['Date'].dt.dayofweek
train_rossman['month'] = train_rossmann['Date'].dt.month


In [27]:
# Split the data into 80% training and 20% validation
train_rossmann, val_rossman = train_rossman.sample(frac=0.8, random_state=42), train_rossman.drop(train_rossman.sample(frac=0.8, random_state=42).index)

In [28]:
import io

csv_train = train_rossmann.to_csv(index=False)

# save training file to S3
s3_client.put_object(Bucket=bucket_name, Key='rossman/train_preprocessed.csv', Body=csv_train)

# Convert the validation set to CSV
csv_val = val_rossman.to_csv(index=False)

# save validation file to S3
s3_client.put_object(Bucket=bucket_name, Key='rossman/val_preprocessed.csv', Body=csv_val)

print("Preprocessed data saved to S3.")


Preprocessed data saved to S3.


In [29]:
# Sample 1000 rows from each dataset and save to new files
train_sample = train_rossmann.sample(n=1000, random_state=42)
test_sample = test_rossmann.sample(n=1000, random_state=42)

# Save the sampled files
train_sample.to_csv("train_sample.csv", index=False)
test_sample.to_csv("test_sample.csv", index=False)


NameError: name 'train_sample' is not defined